In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
from pandas import ExcelWriter

import time, os, sys

# Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

pd.options.display.max_columns = 999
pd.options.display.max_rows = 60
pd.options.display.max_info_rows = 60

In [ ]:
# JSON file of countries
from gtrends import *

In [ ]:
options = webdriver.ChromeOptions()

# Change default download directory to store downloaded files:
prefs = {'download.default_directory' : 'C:/gtrends_files/'}
options.add_experimental_option('prefs', prefs)
options.add_argument("--lang=es")
capabilities = options.to_capabilities()

In [ ]:
BASE_URL = 'https://trends.google.com/trends/'
FILES_LOCATION = 'C:/gtrends_files/'

In [ ]:
# Run Chrome
driver = webdriver.Chrome(desired_capabilities=capabilities)

In [ ]:
# Set geography
geo = 'geo='

# Explore trends (Default 12 months time-frame)
query_term = 'explore?q='

# XPath items
export_button = 'widget-actions-item export'

In [ ]:
def genExcel(fileName, frames):
    dumpExcel = ExcelWriter( str(fileName) + '.xlsx', options={'strings_to_urls': False} )
    for frame in frames:
        frame[0].to_excel(dumpExcel, frame[1])
    dumpExcel.save()
    
    
def query_term_geo(driver, geography, term, timeframe=''):
    # Usage: 
    # query_term_geo(driver=driver, geography='Peru', term='seguridad informatica', timeframe='')
    #
    
    # First usage always return error 429
    driver.get( BASE_URL)
    if timeframe == '':
        driver.get( BASE_URL + query_term + term + '&' + geo + Countries_en[geography] )
    else:
        driver.get( BASE_URL + query_term + term + '&' + geo + Countries_en[geography] + '&' + timeframe)

    time.sleep(4)
    driver.switch_to.window(driver.current_window_handle)
    time.sleep(5)
    download_btn = driver.find_elements_by_xpath('//button[@class="{}"]'.format(export_button))
    download_btn[1].click()
    while os.path.isfile(FILES_LOCATION + 'geoMap.csv') == False:
        time.sleep(3)
    geo_df = pd.read_csv(FILES_LOCATION + 'geoMap.csv', header=1)
    geo_df.columns = ['Region', term]
    os.remove(FILES_LOCATION + 'geoMap.csv')
    
    return geo_df


def multiSearch_term(search_list, delay, geography, csv=False, fileName=''):
    dfs = query_term_geo(driver=driver, geography=geography, term=search_list[0], timeframe='')
    print('Term search: {} ok'.format(search_list[0]))
    for term in search_list[1:]:
        dfs = dfs.merge(query_term_geo(driver=driver, geography=geography, term=term, timeframe=''), on='Region', how='left')
        time.sleep(delay)
        print('Term search: {} ok'.format(term))
    if csv:
        if fileName == '': fileName = 'list_' + time.strftime("%Y-%m-%d_%H", time.gmtime())
        dfs.to_csv(path_or_buf=FILES_LOCATION + fileName + '.csv', encoding='iso-8859-1')
        print('New file created in {}'.format(FILES_LOCATION))
    return dfs


def multiSearch_country(search_list, delay, countries, suffix_file):
    for country in countries:
        print('Current country: {}'.format(country))
        multiSearch_term(search_list=search_list, delay=delay, geography=country, csv=True, fileName=country + suffix_file)
        time.sleep(5)
        print('Ok {}'.format(country))
    print('All countries downloaded!')

In [ ]:
# Example:
mobility_list = ['movilidad', 'laptop', 'ultraslim', 'notebook', 'mobile apps', 'smartphone', 'tablet']

multiSearch_country(search_list=mobility_list, delay=20, countries=['Argentina', 'Chile', 'Colombia', 
                                                                    'Mexico', 'Brazil', 'Peru'], suffix_file='_mobility')